In [ ]:
import torch
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchdata
import torchtext
from torchdata import datapipes
from torchtext import transforms
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
with open('corpus.en_ru.1m.ru', 'r', encoding='utf-8') as file_ru:
    corpus_ru = file_ru.read().splitlines()

with open('corpus.en_ru.1m.en', 'r', encoding='utf-8') as file_en:
    corpus_en = file_en.read().splitlines()

In [ ]:
def preprocessing_text(data):
    data = re.sub(re.compile('<.*?>'), '', data)
    data = re.sub('[^A-Za-zА-Яа-я]+', ' ', data)
    data = data.lower()

    return data

In [ ]:
corpus_ru = [preprocessing_text(seq) for seq in corpus_ru[:10000]]
corpus_en = [preprocessing_text(seq) for seq in corpus_en[:10000]]
print(corpus_ru)

['такое развитие характера гарри может разочаровать читателей полюбивших его былую мстительность но с другой стороны это преображение укрепляет позицию тех кто не видит глубже сюжета и изображения героев ', 'решение суда группа вернулась под крыло к elektra entertainment предотвратило дальнейшие нападки со стороны неугомонного ульриха и не позволило ему обнародовать детали нового контракта ', 'когда тебе или лет легко перенимать бандитские повадки и переносить их в группу ', 'а сейчас куча триьютов тем же самым black sabbath и kiss ', 'я был единственным кто занялся копированием демо на кассете ', 'я вырос при таком свободном воспитании что ты и представить себе не можешь ', 'обычно я просыпался в спускался вниз и видел что дверь в дом открыта на кухне и в гостиной стоит банок пива и дома никого нет ', 'например о смертной казни ', 'в таком звучании нет души человеческая искра отодвинута на задний план ', 'восприятие лишено смысла ', 'эти мысли не обязательно должны быть в очевидном ро

In [ ]:
print(corpus_ru[:10])

['такое развитие характера гарри может разочаровать читателей полюбивших его былую мстительность но с другой стороны это преображение укрепляет позицию тех кто не видит глубже сюжета и изображения героев ', 'решение суда группа вернулась под крыло к elektra entertainment предотвратило дальнейшие нападки со стороны неугомонного ульриха и не позволило ему обнародовать детали нового контракта ', 'когда тебе или лет легко перенимать бандитские повадки и переносить их в группу ', 'а сейчас куча триьютов тем же самым black sabbath и kiss ', 'я был единственным кто занялся копированием демо на кассете ', 'я вырос при таком свободном воспитании что ты и представить себе не можешь ', 'обычно я просыпался в спускался вниз и видел что дверь в дом открыта на кухне и в гостиной стоит банок пива и дома никого нет ', 'например о смертной казни ', 'в таком звучании нет души человеческая искра отодвинута на задний план ', 'восприятие лишено смысла ']


In [ ]:
SOS_token = 1
EOS_token = 2
PAD_token = 0
UNK_token = 3

In [ ]:
tokenized_corpus_ru = [word_tokenize(word) for word in corpus_ru]
tokenized_corpus_en = [word_tokenize(word) for word in corpus_en]

In [ ]:
en_dict = {'<sos>': SOS_token,
          '<eos>': EOS_token,
          '<pad>': PAD_token,
          '<unk>': UNK_token}

ru_dict = {'<sos>': SOS_token,
          '<eos>': EOS_token,
          '<pad>': PAD_token,
          '<unk>': UNK_token}

en_dict_reverse = {SOS_token :'<sos>',
          EOS_token :'<eos>',
          PAD_token : '<pad>',
          UNK_token : '<unk>'}

ru_dict_reverse = {SOS_token :'<sos>',
          EOS_token :'<eos>',
          PAD_token : '<pad>',
          UNK_token : '<unk>'}

In [ ]:
corpus_ru = set([word for subarr in tokenized_corpus_ru for word in subarr])
corpus_en = set([word for subarr in tokenized_corpus_en for word in subarr])

In [ ]:
set_tokenized_corpus_ru = [word for word in corpus_ru]
set_tokenized_corpus_en = [word for word in corpus_en]
set_tokenized_corpus_en

['gently',
 'bases',
 'geforce',
 'supervisory',
 'revising',
 'rwandan',
 'wont',
 'geared',
 'polyphonic',
 'emissary',
 'avoided',
 'easier',
 'encamped',
 'rebecca',
 'inelastic',
 'curry',
 'unincorporated',
 'shift',
 'tbc',
 'schmidt',
 'dome',
 'singles',
 'pica',
 'shifting',
 'traits',
 'shouted',
 'diversify',
 'stages',
 'denunciation',
 'advertisers',
 'windiff',
 'devious',
 'transboundary',
 'loggers',
 'hebrew',
 'fairytale',
 'polycyclic',
 'interfering',
 'scale',
 'towel',
 'tp',
 'pswg',
 'displeased',
 'consulate',
 'lung',
 'undertakes',
 'faber',
 'collar',
 'spent',
 'geneti',
 'repaired',
 'pronko',
 'lucrative',
 'coalitions',
 'gun',
 'reconsider',
 'reboot',
 'special',
 'china',
 'incorrectly',
 'shaliapin',
 'increasing',
 'broken',
 'taima',
 'assr',
 'syntax',
 'apartheid',
 'reintroduced',
 'donated',
 'resonant',
 'cherie',
 'malware',
 'washington',
 'pondering',
 'psp',
 'salt',
 'intimate',
 'injure',
 'nations',
 'sand',
 'saidia',
 'basescu',
 'be

In [ ]:
for i in range(4, len(set_tokenized_corpus_ru) + 4):
    ru_dict[set_tokenized_corpus_ru[i-4]] = i
    ru_dict_reverse[i] = set_tokenized_corpus_ru[i-4]

for i in range(4, len(set_tokenized_corpus_en) + 4):
    en_dict[set_tokenized_corpus_en[i-4]] = i
    en_dict_reverse[i] = set_tokenized_corpus_en[i-4]

In [ ]:
max_size = 0

for i in range(len(tokenized_corpus_ru)):
    for j in range(len(tokenized_corpus_ru[i])):
        tokenized_corpus_ru[i][j] = ru_dict[tokenized_corpus_ru[i][j]]
        if j > max_size:
            max_size = j

for i in range(len(tokenized_corpus_en)):
    for j in range(len(tokenized_corpus_en[i])):
        tokenized_corpus_en[i][j] = en_dict[tokenized_corpus_en[i][j]]
        if j > max_size:
            max_size = j

MAX_SIZE = max_size + 3
print(MAX_SIZE)

66


In [ ]:
def get_dataloader(batch_size, input_sec, output_sec):
    corpus_len = min(len(input_sec), len(output_sec))
    input_ids = np.zeros((corpus_len, MAX_SIZE), dtype=np.int32)
    output_ids = np.zeros((corpus_len, MAX_SIZE), dtype=np.int32)

    for i in range(corpus_len):
        inp_ids = [[SOS_token]]
        input_sec[i].append(EOS_token)
        while len(input_sec[i]) < MAX_SIZE - 2:
            input_sec[i].append(ru_dict['<pad>'])
        inp_ids.append(input_sec[i])
        final_inp_ids = [word for subarr in inp_ids for word in subarr]

        outp_ids = [[SOS_token]]
        output_sec[i].append(EOS_token)
        while len(output_sec[i]) < MAX_SIZE - 2:
            output_sec[i].append(ru_dict['<pad>'])
        outp_ids.append(output_sec[i])
        final_outp_ids = [word for subarr in outp_ids for word in subarr]
        input_ids[i, :len(final_inp_ids)] = torch.LongTensor(final_inp_ids).to(device)
        output_ids[i, :len(final_outp_ids)] = torch.LongTensor(final_outp_ids).to(device)

    dataset = TensorDataset(torch.LongTensor(input_ids).to(device),
                           torch.LongTensor(output_ids).to(device))
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return data_loader

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_k=0.01):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_k)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)

        return output, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_k=0.1, max_length=10):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_k)
        self.SOS_token = 1

    def forward(self, encoder_output, encoder_hidden, target_tensor=None):
        batch_size = encoder_output.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long,
                                    device=device).fill_(self.SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_SIZE):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(decoder_input,
                                                                        decoder_hidden,
                                                                        encoder_output)
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, top = decoder_output.topk(1)
                decoder_input = top.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attn_weights


    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))
        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(input_gru, hidden)
        out = self.out(output)

        return out, hidden, attn_weights

In [ ]:
input_size = len(en_dict)
output_size = len(ru_dict)
print(input_size)
print(output_size)

20148
42066


In [ ]:
encoder = Encoder(input_size, 64).to(device)
decoder = Decoder(64, output_size).to(device)

In [ ]:
def train(data, encoder, decoder, n_epochs=10, learning_rate=0.001):
    encoder_optimizer = optim.Adam(encoder.parameters(), learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), learning_rate)
    criterion = nn.NLLLoss()

    for i in range(1, n_epochs + 1):
        total_loss = 0

        for input_data, target_data in data:
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            encoder_outputs, encoder_hidden = encoder(input_data)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_data)
            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_data.view(-1)
            )
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            total_loss += loss.item()

        print(total_loss / len(data))

In [ ]:
data = get_dataloader(256, tokenized_corpus_en, tokenized_corpus_ru)
train(data, encoder, decoder)

7.154855561256409
3.599385839700699
2.8419729709625243
2.6460356056690215
2.5651125252246856
2.5188179194927214
2.49829785823822
2.4597778499126433
2.454258441925049
2.4341739296913145


In [ ]:
encoder_outputs, encoder_hidden = encoder(torch.LongTensor([[en_dict['cases'], en_dict['like'], en_dict['only']]]).to(device))
decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden)

In [ ]:
decoder_outputs_np = decoder_outputs.argmax(dim=1).cpu().numpy().flatten()
print(decoder_outputs_np)
output = []
i = 0
while decoder_outputs_np[i] != EOS_token:
  if decoder_outputs_np[i] == SOS_token or decoder_outputs_np[i] == PAD_token \
  or decoder_outputs_np[i] == UNK_token:
    i += 1
    continue
  output.append(ru_dict_reverse[decoder_outputs_np[i]])
  i += 1
print(output)

[27  0 14 ...  6 13 12]
['внешними', 'надвигалась', 'geforce', 'психоделическое', 'психоделическое', 'geforce', 'geforce', 'психоделическое', 'geforce', 'психоделическое', 'психоделическое', 'психоделическое', 'психоделическое', 'geforce', 'психоделическое', 'geforce', 'geforce', 'geforce', 'арбитражного', 'внутривенно', 'ханты', 'geforce', 'кинофильмам', 'психоделическое', 'geforce', 'geforce', 'психоделическое', 'психоделическое', 'geforce', 'психоделическое', 'арбитражного', 'geforce', 'geforce', 'geforce', 'geforce', 'психоделическое', 'geforce', 'психоделическое', 'geforce', 'geforce', 'geforce', 'арбитражного', 'geforce', 'психоделическое', 'арбитражного', 'geforce', 'психоделическое', 'geforce', 'geforce', 'geforce', 'психоделическое', 'geforce', 'кинофильмам', 'geforce', 'психоделическое', 'психоделическое', 'ханты', 'психоделическое', 'психоделическое', 'geforce', 'geforce', 'внутривенно', 'психоделическое']


In [ ]:
def translate(sequence):
  encoder_outputs, encoder_hidden = encoder(torch.LongTensor([[en_dict[word]
                                     if word in en_dict else en_dict['<unk>']
                                            for word in sequence]]).to(device))
  decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden)
  decoder_outputs_np = decoder_outputs.argmax(dim=1).cpu().numpy().flatten()
  print(decoder_outputs_np)
  output = []
  i = 0
  while decoder_outputs_np[i] != EOS_token:
    if len(output) >= len(sequence) + 1:
      break
    if decoder_outputs_np[i] == SOS_token or decoder_outputs_np[i] == PAD_token \
    or decoder_outputs_np[i] == UNK_token:
      i += 1
      continue
    output.append(ru_dict_reverse[decoder_outputs_np[i]])
    i += 1

  return output

In [ ]:
print(translate(['i', 'like', 'sport']))

[49  0  5 ...  0  0  3]
['картах', 'квасье', 'кинофильмам', 'кинофильмам']


In [ ]:
print(translate(['he', 'go', 'to', 'the', 'store']))

[51  0  4 ...  1  4  3]
['зрителями', 'кинофильмам', 'кинофильмам', 'кинофильмам', 'кинофильмам', 'кинофильмам']


In [ ]:
print(translate(['Dima', 'want', 'to', 'kill', 'someone']))

[30  0  3 ...  0  3  2]
['расширила']


In [ ]:
print(translate(['cat', 'jump', 'on', 'the', 'table']))

[63  0  5 ...  0  0  4]
['перспектив', 'квасье', 'кинофильмам', 'квасье', 'кинофильмам', 'кинофильмам']


In [ ]:
print(translate(['black', 'is', 'my', 'color']))

[61  0  7 ...  0  6  5]
['ратифицирующие', 'мужчинами', 'квасье', 'почтить', 'кинофильмам']


In [ ]:
print(translate(['you', 'are', 'not', 'me']))

[63  0  7 ...  1  6  5]
['перспектив', 'мужчинами', 'квасье', 'почтить', 'почтить']


In [ ]:
print(translate(['who', 'is', 'dima']))

[50  0  8 ...  1  7  6]
['вводящие', 'анализом', 'почтить', 'мужчинами']


In [ ]:
print(translate(['you', 'know', 'this', 'information', 'is', 'very', 'interesting']))

[42  1  6 ...  0  0  4]
['австралийский', 'почтить', 'кинофильмам', 'квасье', 'квасье', 'кинофильмам', 'кинофильмам', 'квасье']


In [ ]:
print(translate(['i', 'very', 'good']))

[45  0  3 ...  0  3  2]
['иенах']


In [ ]:
print(translate(['wolf', 'bring', 'home']))

[30  0  5 ...  0  5  4]
['расширила', 'квасье', 'кинофильмам', 'квасье']


In [ ]:
len(en_dict)

20148